In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
pip install transformers datasets==3.6.0 evaluate jiwer torchaudio librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 214.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 139.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 217.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 177.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 284.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 210.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 281.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 181.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 280.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 312.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 194.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 240.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

## CV_17

In [ ]:
# import os
# import pandas as pd

# from datasets import Dataset
# df = pd.read_csv("test.tsv", sep='\t')
# df = df[['path','sentence']]

# # Define the folder containing your audio files
# audio_folder = "test_audio"

# # List all files in the folder
# audio_files = set(os.listdir(audio_folder))

# # Filter the DataFrame to include only rows where the 'path' exists in the folder
# df_filtered = df[df['path'].isin(audio_files)]

# # Optionally, save the filtered DataFrame
# df_filtered.to_csv("filtered_dataframe.csv", index=False)
# df_filtered

In [ ]:
# dataset = Dataset.from_pandas(df_filtered)
# dataset

In [3]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

cv_17 = load_dataset("mozilla-foundation/common_voice_17_0", "fr", split="test")

common_voice_17_0.py:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/132k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


n_shards.json:   0%|          | 0.00/17.5k [00:00<?, ?B/s]

...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 12670it [00:00, 126688.64it/s]
Reading metadata...: 32469it [00:00, 134464.92it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13524it [00:00, 135226.91it/s]
Reading metadata...: 28449it [00:00, 143469.50it/s]
Reading metadata...: 43467it [00:00, 146530.08it/s]
Reading metadata...: 63835it [00:00, 144317.31it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 18069it [00:00, 180680.26it/s]
Reading metadata...: 36138it [00:00, 177741.95it/s]
Reading metadata...: 53916it [00:00, 176312.16it/s]
Reading metadata...: 71693it [00:00, 176875.87it/s]
Reading metadata...: 89383it [00:00, 173389.63it/s]
Reading metadata...: 106733it [00:00, 167431.77it/s]
Reading metadata...: 123512it [00:00, 152795.53it/s]
Reading metadata...: 139006it [00:00, 153322.50it/s]
Reading metadata...: 154489it [00:00, 153208.62it/s]
Reading metadata...: 169913it [00:01, 153397.99it/s]
Reading metadata...: 185325it [00:01, 149601.01it/s]
Reading metadata...: 200346it [00:01, 148468.45it/s]
Reading metadata...: 215233it [00:01, 148466.05it/s]
Reading metadata...: 230160it [00:01, 148700.54it/s]
Reading metadata...: 245203it [00:01, 149208.79it/s]
Reading metadata...: 260139it [00:01, 149212.85it/s]
Reading metadata...: 275071it [00:01, 149182.92it/s]
Reading metadata...: 289997it [00:01, 147438.21it/s]
Reading me

In [4]:
dataset = cv_17.remove_columns(['path','client_id','up_votes','down_votes','age','gender','accent','locale','segment','variant'])
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 16159
})

In [5]:
filtered_data = dataset.select(range(1))
def print_first_audio(batch): 
    first_audio = batch["audio"][0] 
    print(f"Sampling rate: {first_audio['sampling_rate']}")
    print(f"array:{first_audio['array']}")
    return batch
filtered_data.map(print_first_audio, batched=True, batch_size=1)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Sampling rate: 48000
array:[ 0.00000000e+00 -4.95903318e-12 -4.28973263e-13 ... -5.87302275e-05
 -8.39358181e-05 -7.38822855e-05]


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1
})

In [6]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [7]:
filtered_data = dataset.select(range(1))
def print_first_audio(batch): 
    first_audio = batch["audio"][0] 
    print(f"Sampling rate: {first_audio['sampling_rate']}")
    print(f"array:{first_audio['array']}")
    return batch
filtered_data.map(print_first_audio, batched=True, batch_size=1)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Sampling rate: 16000
array:[ 1.45519152e-10 -5.82076609e-11 -5.82076609e-11 ... -2.84506677e-05
  2.98366213e-05 -6.71380258e-05]


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1
})

In [8]:
# filtered_data = dataset.select(range(5))

# from datasets import Audio

# filtered_data = filtered_data.cast_column("audio", Audio(sampling_rate=16000))

In [55]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="bofenghuang/whisper-small-cv11-french")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

Device set to use cuda:0


Map:   0%|          | 0/16159 [00:00<?, ? examples/s]

In [56]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: Ce dernier a évolué tout au long de l'histoire romaine.
Transcription: Ce dernier évolue tout au long de l'histoire romaine.
--------------------------------------------------
Ground Truth: Son actionnaire majoritaire est le Conseil territorial de Saint-Pierre-et-Miquelon.
Transcription: Son actionnaire majoritaire est le conseil territorial de Saint-Pierre-et-Miquelon.
--------------------------------------------------
Ground Truth: Ce site contient quatre tombeaux de la dynastie achéménide et sept des Sassanides.
Transcription: Ce site contient quatre tombeaux de la dynastie Hachéménide et sept des Sassanides.
--------------------------------------------------
Ground Truth: J'ai dit que les acteurs de bois avaient, selon moi, beaucoup d'avantages sur les autres.
Transcription: J'ai dit que les acteurs de bois avaient, selon moi, beaucoup d'avantage sur les autres.
--------------------------------------------------
Ground Truth: Les Pays-Bas ont remporté toutes les éditi

In [57]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="nambn0321/ASR_french_3")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

# Now transcribed_dataset contains: 'audio', 'sentence', 'transcription'


Device set to use cuda:0


Map:   0%|          | 0/16159 [00:00<?, ? examples/s]

In [58]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: Ce dernier a évolué tout au long de l'histoire romaine.
Transcription: Ce dernier est volé tout au long de l'histoire romaine.
--------------------------------------------------
Ground Truth: Son actionnaire majoritaire est le Conseil territorial de Saint-Pierre-et-Miquelon.
Transcription: Son actionnaire majoritaire est le conseil territorial de Saint-Pierre-et-Miculon.
--------------------------------------------------
Ground Truth: Ce site contient quatre tombeaux de la dynastie achéménide et sept des Sassanides.
Transcription: Ce site contient quatre tombeaux de la dynastie Hachéménide et sept des Sassanides.
--------------------------------------------------
Ground Truth: J'ai dit que les acteurs de bois avaient, selon moi, beaucoup d'avantages sur les autres.
Transcription: J’ai dit que les acteurs de bois avaient, selon moi, beaucoup d’avantages sur les autres.
--------------------------------------------------
Ground Truth: Les Pays-Bas ont remporté toutes les édi

In [9]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-medium")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

# Now transcribed_dataset contains: 'audio', 'sentence', 'transcription'


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Map:   0%|          | 0/16159 [00:00<?, ? examples/s]

`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [10]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: Ce dernier a évolué tout au long de l'histoire romaine.
Transcription:  Ce dernier a évolué tout au long de l'histoire romaine.
--------------------------------------------------
Ground Truth: Son actionnaire majoritaire est le Conseil territorial de Saint-Pierre-et-Miquelon.
Transcription:  Son actionnaire majoritaire est le conseil territoriale de Saint-Pierre-et-Miquelon.
--------------------------------------------------
Ground Truth: Ce site contient quatre tombeaux de la dynastie achéménide et sept des Sassanides.
Transcription:  Ce site contient quatre tombeaux de la dynastie Hasheménide et sept de Sassanide.
--------------------------------------------------
Ground Truth: J'ai dit que les acteurs de bois avaient, selon moi, beaucoup d'avantages sur les autres.
Transcription:  J'ai dit que les acteurs de bois avaient, selon moi, beaucoup d'avantages sur les autres.
--------------------------------------------------
Ground Truth: Les Pays-Bas ont remporté toutes les

## CV_16

In [32]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

cv_16 = load_dataset("mozilla-foundation/common_voice_16_0", "fr", split="test")

DatasetNotFoundError: Dataset 'mozilla-foundation/common_voice_16_0' is a gated dataset on the Hub. Visit the dataset page at https://huggingface.co/datasets/mozilla-foundation/common_voice_16_0 to ask for access.

In [30]:
dataset = cv_16.remove_columns(['path','client_id','up_votes','down_votes','age','gender','accent','locale','segment','variant'])
dataset

NameError: name 'cv_16' is not defined

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

## CV_15

In [34]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

cv_15 = load_dataset("mozilla-foundation/common_voice_15_0", "fr", split="test")

DatasetNotFoundError: Dataset 'mozilla-foundation/common_voice_15_0' is a gated dataset on the Hub. Visit the dataset page at https://huggingface.co/datasets/mozilla-foundation/common_voice_15_0 to ask for access.

In [ ]:
dataset = cv_15.remove_columns(['path','client_id','up_votes','down_votes','age','gender','accent','locale','segment','variant'])
dataset

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

## MLS

In [11]:
from datasets import load_dataset

# Try loading the dataset without streaming
dataset = load_dataset("facebook/multilingual_librispeech", "french",split="test")

dataset

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/34 [00:00<?, ?it/s]

french/dev-00000-of-00001.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

french/test-00000-of-00001.parquet:   0%|          | 0.00/158M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/2426 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/258213 [00:00<?, ? examples/s]

Generating 9_hours split:   0%|          | 0/2167 [00:00<?, ? examples/s]

Generating 1_hours split:   0%|          | 0/241 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'original_path', 'begin_time', 'end_time', 'transcript', 'audio_duration', 'speaker_id', 'chapter_id', 'file', 'id'],
    num_rows: 2426
})

In [12]:
dataset = dataset.remove_columns(['original_path', 'begin_time', 'end_time', 'audio_duration', 'chapter_id', 'file', 'id','speaker_id'])
dataset

Dataset({
    features: ['audio', 'transcript'],
    num_rows: 2426
})

In [13]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [14]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcript"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

In [15]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcript']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: pendant le second siècle je fis serment d'ouvrir tous les trésors de la terre à quiconque me mettrait en liberté mais je ne fus pas plus heureux dans le troisième je promis de faire puissant monarque mon libérateur d'être toujours près de lui en esprit
Transcription:  Pendant le second siècle, je fis serment d'ouvrir tous les trésors de la terre, à qui compte me mettre en liberté. Mais je ne suis pas plus heureux. Dans le troisième, je promis de faire puissant mon arc, mon libérateur, d'être toujours près de lui en esprit.
--------------------------------------------------
Ground Truth: non ta mort est certaine dit le génie choisis seulement de quelle sorte tu veux que je te fasse mourir le pêcheur le voyant dans la résolution de le tuer en eut une douleur extrême non pas tant pour l'amour de lui qu'à cause de ses trois enfants dont il plaignait la misère où ils allaient être réduits par sa mort
Transcription:  — Non, ta mort est certaine, dit le génie. Choisis seulement 

In [16]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="nambn0321/ASR_french_3")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcript"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

Device set to use cuda:0


Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProce

In [17]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcript']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: pendant le second siècle je fis serment d'ouvrir tous les trésors de la terre à quiconque me mettrait en liberté mais je ne fus pas plus heureux dans le troisième je promis de faire puissant monarque mon libérateur d'être toujours près de lui en esprit
Transcription: Pendant le second siècle, je fis serment d’ouvrir tous les trésors de la terre à qui compte me mettre en liberté, mais je ne fus pas plus heureux.Dans le troisième, je promis de faire puissant mon arc, mon libérateur, d’être toujours près de lui en esprit.
--------------------------------------------------
Ground Truth: non ta mort est certaine dit le génie choisis seulement de quelle sorte tu veux que je te fasse mourir le pêcheur le voyant dans la résolution de le tuer en eut une douleur extrême non pas tant pour l'amour de lui qu'à cause de ses trois enfants dont il plaignait la misère où ils allaient être réduits par sa mort
Transcription: Non! ta mort est certaine, dit le génie, choisissez seulement de q

In [18]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="bofenghuang/whisper-small-cv11-french")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcript"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/484M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

In [19]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcript']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: pendant le second siècle je fis serment d'ouvrir tous les trésors de la terre à quiconque me mettrait en liberté mais je ne fus pas plus heureux dans le troisième je promis de faire puissant monarque mon libérateur d'être toujours près de lui en esprit
Transcription: Pendant le second siècle, je fis serment d'ouvrir tous les trésors de la Terre, à qui compte mes maîtresses en liberté, mais je ne fus pas plus heureux.Dans le troisième, je promis de faire puissant monarque, mon libérateur, d'être toujours près de lui en esprit.
--------------------------------------------------
Ground Truth: non ta mort est certaine dit le génie choisis seulement de quelle sorte tu veux que je te fasse mourir le pêcheur le voyant dans la résolution de le tuer en eut une douleur extrême non pas tant pour l'amour de lui qu'à cause de ses trois enfants dont il plaignait la misère où ils allaient être réduits par sa mort
Transcription: Non, ta mort est certaine, dit le génie, choisir seulement 

In [20]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-medium")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcript"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

Device set to use cuda:0


Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

In [21]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcript']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: pendant le second siècle je fis serment d'ouvrir tous les trésors de la terre à quiconque me mettrait en liberté mais je ne fus pas plus heureux dans le troisième je promis de faire puissant monarque mon libérateur d'être toujours près de lui en esprit
Transcription:  Pendant le second siècle, je fis serment d'ouvrir tous les trésors de la terre, à quiconque me mettrait en liberté, mais je ne fus pas plus heureux. Dans le troisième, je promis de faire puissant monarque mon libérateur, d'être toujours près de lui en esprit,
--------------------------------------------------
Ground Truth: non ta mort est certaine dit le génie choisis seulement de quelle sorte tu veux que je te fasse mourir le pêcheur le voyant dans la résolution de le tuer en eut une douleur extrême non pas tant pour l'amour de lui qu'à cause de ses trois enfants dont il plaignait la misère où ils allaient être réduits par sa mort
Transcription:  — Non, ta mort est certaine, dit le génie. Choisis seulement 

## Fleurs

In [22]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

fleurs = load_dataset("google/fleurs", "fr_fr", split="test")
fleurs

README.md: 0.00B [00:00, ?B/s]

fleurs.py: 0.00B [00:00, ?B/s]

The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


data/fr_fr/audio/train.tar.gz:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

data/fr_fr/audio/dev.tar.gz:   0%|          | 0.00/143M [00:00<?, ?B/s]

...

test.tsv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 676
})

In [24]:
dataset = fleurs.remove_columns(['id', 'num_samples', 'path', 'raw_transcription', 'gender', 'lang_id', 'lang_group_id','language'])
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 676
})

In [26]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcription"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcriptions": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

Device set to use cuda:0


Map:   0%|          | 0/676 [00:00<?, ? examples/s]

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcription']
    predicted_text = transcription['transcriptions']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcription"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcriptions": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcription']
    predicted_text = transcription['transcriptions']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcription"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcriptions": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcription']
    predicted_text = transcription['transcriptions']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["transcription"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcriptions": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['transcription']
    predicted_text = transcription['transcriptions']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)